Confidence <br/>
![](http://www.sciweavers.org/upload/Tex2Img_1651907171/render.png)

<br/>
Assume:
Data is normal

# Imports & loading

In [2]:
from statistics import *

import numpy as np
import scipy
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_dark"

pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 100)
pd.set_option("plotting.backend", "plotly")

In [54]:
df_original = pd.read_csv("exp_survey.csv", encoding="utf8")

# Cleaning

In [55]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 76 columns):
 #   Column                                                                                              Non-Null Count  Dtype  
---  ------                                                                                              --------------  -----  
 0   Timestamp                                                                                           30 non-null     object 
 1   الجنس                                                                                               30 non-null     object 
 2   التخصص                                                                                              30 non-null     object 
 3   الدفعة                                                                                              30 non-null     object 
 4   أعلى درجة قدرات لك (مثلا: "83")                                                                     30 non-null     in

In [157]:
df = pd.read_csv("exp_survey.csv", encoding="utf8")

# Manual editing, yikes!
df.at[9, "قدر عدد الجهات اللتي قدمت عليها"] = "15"
df.at[20, "قدر عدد الجهات اللتي قدمت عليها"] = "15"
df.at[20, "قدر عدد الجهات اللتي ردت عليك"] = "6"
df.at[20, "قدر عدد الجهات اللتي ردت عليك بالرفض"] = "4"
df.at[25, "مساحة تعليقات حرة.3"] = np.nan

# Macro filtering
df = (df
    .drop("Timestamp", axis=1)
    .dropna(how="all", axis=1) # Drops empty columns
)

# Columns breakdown 
comments_cols = [col for col in df.columns if "مساحة" in col or "نصائح" in col]
categorical_cols = [col for col in df.columns if "هل" in col or "كيف" in col]
categorical_cols.extend(["الجنس", "التخصص", "الدفعة", "بعد أي مستوى كان التدريب", "كم كان الأجر بالشهر (بالآف)"])
checkbox_cols = [
    "ماهي المجالات المهتم فيها",
    "كيف طبيعة التطوع",
    "في أي المجالات التالية كانت مشاريعك",
    "ما نوع الاسئلة في المقابلة",
    "في أي المجالات التالية كانت تدريبك"
]
number_cols = set(df.columns).difference(set((*checkbox_cols, *categorical_cols, *comments_cols)))


for col in categorical_cols:
    df[col] = df[col].astype("category")


# Expand checkbox cols to multiple boolean cols
for col in checkbox_cols:
    categories = set(val.strip() for val in (",".join(df[col].dropna().values)).split(","))
    for category in categories:
        new_col = f"{col} [{category}]"
        df[new_col] = df[col].apply(lambda x: np.NaN if not isinstance(x, str) else category in x)
    df = df.drop(col, axis=1)


for col in number_cols:
    df[col] = df[col].astype("float32")

In [158]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 76 columns):
 #   Column                                                                                              Non-Null Count  Dtype  
---  ------                                                                                              --------------  -----  
 0   Timestamp                                                                                           30 non-null     object 
 1   الجنس                                                                                               30 non-null     object 
 2   التخصص                                                                                              30 non-null     object 
 3   الدفعة                                                                                              30 non-null     object 
 4   أعلى درجة قدرات لك (مثلا: "83")                                                                     30 non-null     in

In [159]:
df[comments_cols]

,مساحة تعليقات حرة.1,اذا كان جوابك للسؤال السابق نعم، أكتب نصائح/موارد في انتاج سيرة ذاتية جيدة,مساحة تعليقات حرة.4,نصائح قبل المقابلة,مساحة تعليقات حرة.5
0,NaN,استخدم ليتيك,NaN,NaN,NaN
1,NaN,Latex,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN
8,NaN,خلال المقابلات لاحظت ان الشركات تركز على عدد ا...,اغلب المقابلات كانو ما يشوفون github بس كانو ي...,NaN,اختر مكان تدريبك بعنايه لانه غالبا بيكون المكا...
9,NaN,اضافة المشاريع التي عملت عليها خارج الجامعة وا...,NaN,NaN,NaN
